In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
import time
import os
import copy
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.utils.data as Data
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
import time
import pathlib
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split



In [2]:
#checking for device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(device)

In [3]:
#Transforms
transform_data={
    "trainData":transforms.Compose([transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(p=0.5),  # horizontal flip and 0.5 is the position
    transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.RandomRotation(45, resample=False, expand=False,
                                              center=None),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    #transforms.Normalize([0.5],[0.5]) # 0-1 to [-1,1] , formula (x-mean)/std

]),
    "testData": transforms.Compose([ transforms.Grayscale(num_output_channels=1),      
        transforms.RandomHorizontalFlip(p=0.5),  # horizontal flip and 0.5 is the position
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        #transforms.Normalize([0.5],[0.5])
    ])
}

data_transform={
    "trainData":transforms.Compose([transforms.Grayscale(num_output_channels=1), 
    transforms.RandomHorizontalFlip(p=0.5),  # horizontal flip and 0.5 is the position
    transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.RandomRotation(45, resample=False, expand=False,
                                              center=None),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    #transforms.Normalize([0.5],[0.5]) # 0-1 to [-1,1] , formula (x-mean)/std
                            ])        }

In [4]:
#Dataloader
batchSize = 96 
#Path for training and testing directory
train_path='C:/automated_classification/holographic_plankton_classification-main/samples/new_training'

# Split test and train dataset 
#data_transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
#                                     transforms.ToTensor()])
dataset = ImageFolder(train_path, transform=data_transform["trainData"])
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])

train_loader=DataLoader(
    train_data,
    batch_size=batchSize, shuffle=True
)
test_loader=DataLoader(
    test_data,
    batch_size=batchSize, shuffle=True)

#train_path='C:/automated_classification/holographic_plankton_classification-main/samples/training_new/train_data'
#test_path='C:/automated_classification/holographic_plankton_classification-main/samples/training_new/test_data'

#train_loader=DataLoader(
 #   torchvision.datasets.ImageFolder(train_path,transform=transform_data["trainData"]),
 #   batch_size=batchSize, shuffle=True
#)
#test_loader=DataLoader(
 #   torchvision.datasets.ImageFolder(test_path,transform=transform_data["testData"]),
  #  batch_size=batchSize, shuffle=True
#)


In [5]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)
numClasses=len(classes)
print(numClasses)
print(len(test_loader.dataset))

['Decoy', 'Kbrevis', 'NullClass-autoholo']
3
540


In [6]:
#featureExtract=False
usePretrained=True
modelName = "vgg19"
dropNum = 0.5
#momentum = 0.9
dropout = True
mono = True
filePath = "C:/automated_classification/holographic_plankton_classification-main/"
savePath = 'C:/automated_classification/holographic_plankton_classification-main/modelGen/' + modelName + '_'

def setParameterRequiresGrad(model, featureExtract):
    if featureExtract:
        for param in model.parameters():
            param.requires_grad = False


def initialModel(modelName, numClasses, featureExtract, usePretrained):
    if modelName == "vgg19":
        modelUse = models.vgg19(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        modelUse.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=numClasses, bias=True)

    elif modelName == "shufflenet_v2_x2_0":
        modelUse = models.shufflenet_v2_x2_0(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.conv1._modules['0'] = nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        if dropout:
            modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                        nn.Linear(2048, numClasses, bias=True))
        else:
            modelUse.fc = nn.Linear(2048, numClasses, bias=True)

    elif modelName == "shufflenet_v2_x0_5":
        modelUse = models.shufflenet_v2_x0_5(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.conv1._modules['0'] = nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        if dropout:
            modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                        nn.Linear(1024, numClasses, bias=True),
                                        )
        else:
            modelUse.fc = nn.Linear(1024, numClasses, bias=True)


    elif modelName == "shufflenet_v2_x1_5":
        modelUse = models.shufflenet_v2_x1_5(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.conv1._modules['0'] = nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        if dropout:
            modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                        nn.Linear(1024, numClasses, bias=True),)
        else:
            modelUse.fc = nn.Linear(1024, numClasses, bias=True)


    elif modelName == "shufflenet_v2_x1_0":
        modelUse = models.shufflenet_v2_x1_0(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.conv1._modules['0'] = nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        if dropout:
            modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                        nn.Linear(1024, numClasses, bias=True))
        else:
            modelUse.fc = nn.Linear(1024, numClasses, bias=True)


    elif modelName == "squeezenet1_1":
        modelUse = models.squeezenet1_1(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2))

        modelUse.classifier = nn.Sequential(nn.Dropout(p=dropNum, inplace=False),
                                nn.Conv2d(512, numClasses, kernel_size=(1, 1), stride=(1, 1)),
                                nn.ReLU(inplace=True),
                                nn.AdaptiveAvgPool2d(output_size=(1, 1)))



    elif modelName == "densenet121":
        modelUse = models.densenet121(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.features._modules['conv0'] = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        if dropout:
            modelUse.classifier = nn.Sequential(nn.Dropout(dropNum),
                                        nn.Linear(1024, numClasses, bias=True))
        else:
            modelUse.classifier = nn.Linear(1024, numClasses, bias=True)


    elif modelName == "vgg16":
        modelUse = models.vgg16(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        modelUse.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=numClasses, bias=True)

    elif modelName == 'mobilenet_v2':
        modelUse = models.mobilenet_v2(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.features._modules['0']._modules['0'] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2),
                                                                      padding=(1, 1), bias=False)

        modelUse.classifier = nn.Sequential(
            nn.Dropout(p=dropNum, inplace=False),
            nn.Linear(in_features=1280, out_features=numClasses, bias=True)
        )

    elif modelName == "resnet18":
        modelUse = models.resnet18(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        numFeatures = modelUse.fc.in_features
        if mono:
            modelUse.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        if dropout:
            modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                        nn.Linear(numFeatures, numClasses))
        else:
            modelUse.fc = nn.Linear(numFeatures, numClasses)

    elif modelName == "resnet34":
        modelUse = models.resnet34(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        numFeatures = modelUse.fc.in_features
        if mono:
            modelUse.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        if dropout:
            modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                        nn.Linear(numFeatures, numClasses))
        else:
            modelUse.fc = nn.Linear(numFeatures, numClasses)

    elif modelName == "resnet50":
        modelUse = models.resnet50(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        numFeatures = modelUse.fc.in_features
        if mono:
            modelUse.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        if dropout:
            modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                        nn.Linear(numFeatures, numClasses))
        else:
            modelUse.fc = nn.Linear(numFeatures, numClasses)
            
    elif modelName == "alexnet":
        modelUse = models.alexnet(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
        modelUse.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=numClasses, bias=True) 
        
    elif modelName == "googlenet":
        modelUse = models.googlenet(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.conv1._modules['conv']=nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        modelUse.fc = nn.Linear(in_features=1024, out_features=numClasses, bias=True)    

    else:
        print("model not implemented")
        return None, None

    return modelUse


In [8]:
savePreAndRealFlag = True
num_epochs=30
learningRate=0.0001

def trainModel(model, trainLoader, testLoader, loss_function, optimizer, num_epochs):
    bestAccuracy = 0
    bestModuleWeight = copy.deepcopy(model.state_dict())
    train_accuracy = []
    train_loss = []
    test_accuracy = []
    test_loss = []
    
    for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
        model.train()
        run_accuracy=0.0
        run_loss=0.0
    
        for images,labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            
            outputs=model(images)
            loss=loss_function(outputs,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        
            run_loss+= loss.item() * images.size(0)
            _,prediction=torch.max(outputs.data,1)
        
            run_accuracy+=int(torch.sum(prediction==labels.data))
        
        epoch_accuracy=run_accuracy/len(train_loader.dataset)
        epoch_loss=run_loss/len(train_loader.dataset)
        print("Epoch: {} Phase: {} loss: {}, acc: {}".format(epoch, 'train', epoch_loss, epoch_accuracy))

        train_loss.append(epoch_loss)
        train_accuracy.append(epoch_accuracy)
    
    # Evaluation on testing dataset
        run_accuracy=0.0
        run_loss=0.0
        model.eval()
        if savePreAndRealFlag:
            preResult = []
            realResult = []
            
        for images,labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs=model(images)
            run_loss+= loss.item() * images.size(0)
            _,prediction=torch.max(outputs.data,1)
            run_accuracy+=int(torch.sum(prediction==labels.data))
            
            if epoch>num_epochs-5:
                if savePreAndRealFlag:
                    preResult.append(str(prediction.view(-1).cpu().numpy()))
                    realResult.append(str(labels.view(-1).cpu().numpy()))
    
        epoch_accuracy=run_accuracy/len(test_loader.dataset)
        epoch_loss=run_loss/len(test_loader.dataset)
        print("Epoch: {} Phase: {} loss: {}, acc: {}".format(epoch, 'test', epoch_loss, epoch_accuracy))
        
        if epoch_accuracy > bestAccuracy:
            bestAccuracy = epoch_accuracy
            bestModuleWeight = copy.deepcopy(model.state_dict())


        test_loss.append(epoch_loss)
        test_accuracy.append(epoch_accuracy)
    
    if epoch >num_epochs-5:
        if savePreAndRealFlag:
            preFile = open(filePath +'fileOutput/' + modelName + time.strftime('%m%d%H%M')+ '_wb12_pre.txt', 'w')
            preFile.write(str(preResult))
            realFile = open(filePath+'fileOutput/' + modelName + time.strftime('%m%d%H%M')+ '_wb12_real.txt', 'w')
            realFile.write(str(realResult))
            preFile.close()
            realFile.close()
    model.load_state_dict(bestModuleWeight)
    
    return model, train_loss, train_accuracy, test_loss, test_accuracy
   
    
   
    
    

In [ ]:
modelUse = initialModel(modelName, numClasses, featureExtract=False, usePretrained=False)
modelUse = modelUse.to(device)
optimizer = torch.optim.Adam(modelUse.parameters(), lr=learningRate)
loss_function = nn.CrossEntropyLoss()

modelReturn, trHis, trAcc, teHis, teAcc = trainModel(modelUse, train_loader, test_loader, loss_function, optimizer, 
                                                     num_epochs)
torch.save(modelReturn, savePath + time.strftime('%m%d%H%M') + 'wb12_res01_batch.pt')
torch.save(modelReturn.state_dict, savePath + time.strftime('%m%d%H%M') + '_Para_res01_batch.pt')


trainaccFile = open(savePath+ time.strftime('%m%d%H%M')+ '_trainacc.txt', 'w')
trainaccFile.write(str(trAcc).replace("[", "").replace("]", "").replace(",", "")) 
trainaccFile.close()
trainlossFile = open(savePath+ time.strftime('%m%d%H%M')+ '_trainloss.txt', 'w')
trainlossFile.write(str(trHis).replace("[", "").replace("]", "").replace(",", "")) 
trainlossFile.close()
testlossFile = open(savePath+ time.strftime('%m%d%H%M')+ '_testloss.txt', 'w')
testlossFile.write(str(teHis).replace("[", "").replace("]", "").replace(",", "")) 
testlossFile.close()
testaccFile = open(savePath+ time.strftime('%m%d%H%M')+ '_testacc.txt', 'w')
testaccFile.write(str(teAcc).replace("[", "").replace("]", "").replace(",", "")) 
testaccFile.close()

saveparamsFile = open(savePath+ time.strftime('%m%d%H%M%S')+ '_playingwithparams.txt', 'w')
saveparamsFile.write(str(modelName) +" LearningRate = " + str(learningRate) +" featureExtract = " + str(featureExtract) +" Epochnumber = " + str(num_epochs) +" batchsize = " + str(batchSize) )
saveparamsFile.close()

torch.cuda.empty_cache()

Epoch: 0 Phase: train loss: 1.0564613262812297, acc: 0.4115740740740741
Epoch: 0 Phase: test loss: 1.0729354619979858, acc: 0.3814814814814815
Epoch: 1 Phase: train loss: 0.8287811279296875, acc: 0.49444444444444446
Epoch: 1 Phase: test loss: 0.5808207988739014, acc: 0.8537037037037037
Epoch: 2 Phase: train loss: 0.4312287539243698, acc: 0.8194444444444444
Epoch: 2 Phase: test loss: 0.20006828010082245, acc: 0.8444444444444444
Epoch: 3 Phase: train loss: 0.3967915819750892, acc: 0.8356481481481481
Epoch: 3 Phase: test loss: 0.29481241106987, acc: 0.825925925925926
Epoch: 4 Phase: train loss: 0.2931522594557868, acc: 0.8689814814814815
Epoch: 4 Phase: test loss: 0.18616998195648193, acc: 0.8833333333333333
Epoch: 5 Phase: train loss: 0.9512595845593347, acc: 0.6893518518518519
Epoch: 5 Phase: test loss: 0.8599586486816406, acc: 0.45555555555555555
Epoch: 6 Phase: train loss: 0.5588202966584099, acc: 0.7162037037037037
Epoch: 6 Phase: test loss: 0.4783421754837036, acc: 0.857407407407407

In [38]:
for images,labels in train_loader:
            images, labels = images.to(device), labels.to(device)
index = images[2].cpu().data.numpy()
print(index)
plt.imshow(index[0])

In [80]:
images,labels = test_loader.dataset
print(images.shape)
plt.imshow( images.permute(1, 2, 0)  )
print(len(images))
print(len(labels))

ValueError: too many values to unpack (expected 2)

In [18]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
import torchvision.models as models
modelUse = models.densenet201(pretrained=False)

print(modelUse)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [7]:
  elif modelName == "inception_v3":
        modelUse = models.inception_v3(pretrained=usePretrained)
        setParameterRequiresGrad(modelUse, featureExtract)
        if mono:
            modelUse.Conv2d_1a_3x3._modules['conv']=nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        modelUse.fc = nn.Linear(in_features=2048, out_features=numClasses, bias=True)     

SyntaxError: invalid syntax (<ipython-input-7-5da3443e40a5>, line 1)

In [8]:
elif modelName == "densenet161":
                modelUse = models.densenet161(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.features._modules['conv0'] = nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

                if dropout:
                    modelUse.classifier = nn.Sequential(nn.Dropout(dropNum),
                                                nn.Linear(2208, numClasses, bias=True))
                else:
                    modelUse.classifier = nn.Linear(2208, numClasses, bias=True)

SyntaxError: invalid syntax (<ipython-input-8-270a845accc8>, line 1)

In [7]:
import torch.nn as nn
from torchvision import datasets, transforms, models
featureExtract=True
mono=True
numClasses=17
def setParameterRequiresGrad(model, featureExtract):
    if featureExtract:
        for param in model.parameters():
            param.requires_grad = False
modelUse = models.vgg19(pretrained=False)
#modelUse.Conv2d_1a_3x3._modules['conv']=nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
#modelUse.fc = nn.Linear(in_features=2048, out_features=3, bias=True)
setParameterRequiresGrad(modelUse, featureExtract)
if mono:
    modelUse.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
modelUse.classifier._modules['0'].requires_grad=True
modelUse.classifier._modules['3'].requires_grad=True
modelUse.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=numClasses, bias=True)
print(modelUse)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd